In [1]:
import numpy as np
import pandas as pd
import sklearn
import time
from sklearn.model_selection import train_test_split
import matplotlib.pylab as plt
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import sys
import numpy
import os
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
for date in ["8_18"]: # , "9_02", "9_09", "9_20"
  for img_size in [64]:
    # load the data
    data = np.load(f'twri_rgb_6832_cotton_{img_size}x{img_size}_{date}.npy')
    labels = pd.read_csv('labels_rgb.csv')
    y_label = labels['class'].values

    # remove the filler type
    new_data_without_filler = np.zeros((5376, img_size, img_size, 3)) # defult was 32,32,3
    new_y_label_without_filler = np.zeros(5376, dtype=int)
    i = 0
    for idx, val in enumerate(y_label):
      if val != 0:
        new_data_without_filler[i] = data[idx]
        new_y_label_without_filler[i] = y_label[idx]
        i+=1
    data = new_data_without_filler
    y_label = new_y_label_without_filler - 1

    # split the data and normalization
    train_images, test_images, train_labels, test_labels = train_test_split(data, y_label, test_size=0.3, random_state=42)
    train_images, test_images = train_images / 255.0, test_images / 255.0
    class_names = ['rainfed', 'fully irrigated', 'percent deficit', 'time delay']

    # set up the model and train
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(4))
    # model.summary()
    # compile the model
    model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

    history = model.fit(train_images, train_labels, epochs=70, 
                      validation_data=(test_images, test_labels))